In [1]:
import torch
from datasets import load_dataset
from datasets import load_from_disk
import random, math


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
#定义数据集
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, split):
        # dataset = load_dataset(path='lansinuote/ChnSentiCorp', split=split)
        dataset = load_from_disk('./data/ChnSentiCorp')['%s' % split]
        
        def f(data):
            return len(data['text']) > 20
        self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset['text'][i]
        
        # print('原始数据: ', text)
        # label = self.dataset['label'][i]
        #切分一句话为前半句和后半句
        s_len = len(text)
        half_len = math.floor(s_len / 2)  # 切分句子为两段
        
        sentence1 = text[:half_len]
        sentence2 = text[half_len: ]
        label = 0

        # 有一半的概率把后半句替换为一句无关的话
        if random.randint(0, 1) == 0:
            j = random.randint(0, len(self.dataset) - 1)
            # sentence2 = self.dataset[j]['text'][20:40]
            sentence2 = self.dataset['text'][j][half_len:]
            label = 1

        return sentence1, sentence2, label

# 加载
dataset = Dataset('train')

Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/train/cache-2c893bcab2dc48fd.arrow


In [7]:
for i in range(2):
    print(dataset[i])
    print("")

('选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般', '', 1)

('15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜', '夹杂着人物之间的责任，关心和感情纠葛。一层又一层，惊险刺激。女主角谢怀珉的医术，的确相当专业。看来在现代，学的中医派上用场，成为她的铺垫。不过，书的最后，简直太意外了。都不希望是真的。书中的慧空说，她可以母仪天下。我就猜想，如果她爱的人士个君王，那就很有可能了。希望真是如此呢。', 1)



In [8]:
# 测试拼接的数据

sentence1, sentence2, label = dataset[3]

len(dataset), sentence1, sentence2, label

(9552, '今天才知道这书还有第6卷,真有点郁闷:为什么同一套书有两种版本呢?当当', '动程序、使用说明放在了E盘中。电池有延保卡。', 1)

In [9]:
from transformers import BertTokenizer
from transformers import BertModel


# 加载字典和分词工具
token = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path='bert-base-chinese',
    force_download=False,
)

print(token)


# 加载预训练模型
pretrained = BertModel.from_pretrained(
    pretrained_model_name_or_path='bert-base-chinese',
    force_download=False
).to(device)

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def collate_fn(data):
    sents = [i[:2] for i in data]  # 句子一, 句子二
    labels = [i[2] for i in data]

    # print('sents: ', sents)
    
    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=200,
                                   return_tensors='pt',
                                   return_length=True,
                                   add_special_tokens=True)

    # input_ids: 编码之后的数字
    # attention_mask: 是补零的位置是0,其他位置是1
    # token_type_ids: 第一个句子和特殊符号的位置是0,第二个句子的位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, labels

## 数据加载器

In [11]:

loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=8,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)



for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(loader):
    print(len(loader))
    print(token.decode(input_ids[0]))
    input_ids.shape, attention_mask.shape, token_type_ids.shape, labels
    break

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


1194
[CLS] 以 前 去 过 大 连 ， 今 年 选 择 住 在 博 览 ， 总 体 感 觉 还 是 挺 好 的 ， 就 是 住 在 这 里 的 日 本 人 挺 多 的 ， 最 讨 厌 小 日 本 了 。 我 个 人 习 惯 住 酒 店 最 在 意 的 是 卫 生 ， 尤 其 卫 生 间 的 卫 生 最 重 要 ， 这 里 的 卫 生 条 件 还 是 让 人 满 意 的 ， 服 务 也 不 像 说 的 那 么 不 好 ， 服 务 人 员 态 度 都 很 热 情 友 好 ， [SEP] 停 水 等 待 它 慢 慢 流 走 ， 如 果 一 味 痛 快 洗 澡 就 意 味 着 脚 脖 子 被 泡 沫 淹 掉 。 赶 紧 改 善 吧 ～ [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [12]:
#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)

In [13]:
# 定义下游任务模型
class Model(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                             attention_mask=attention_mask,
                             token_type_ids=token_type_ids)
            pass
        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out

In [14]:
model = Model()
# model.load_state_dict(torch.load('chinese_infer_mission_2023_4_10.pt'))
# chinese_infer_mission_2023_4_20_v1
model.load_state_dict(torch.load('chinese_infer_mission_2023_4_20_v4.pt'))

# 加载模型

# 模型转移到GPU上
model.to(device)

Model(
  (fc): Linear(in_features=768, out_features=2, bias=True)
)

In [15]:
# 句子测试封装


def str_felling_detect(model, str_sents):
    
    out = token.encode_plus(str_sents,
                            truncation=True,
                            padding='max_length',
                            max_length=45,
                            return_tensors='pt',
                            return_length=True,
                            add_special_tokens=True)
    # print(out)
    input_ids = out['input_ids'].to(device)
    attention_mask = out['attention_mask'].to(device)
    token_type_ids = out['token_type_ids'].to(device)
    
    out_test = model(input_ids=input_ids,
                     attention_mask=attention_mask,                 
                     token_type_ids=token_type_ids
                )
    
    t1 = out_test[0][0].item()
    t2 = out_test[0][1].item()
    
    # print('相关性: %.4f \t 不相关性: %.4f' % (t1, t2))
    y_head = out_test.argmax(dim=1).cpu().item()
    
    # y_head: 1: 句子不相关. 0: 相关
    return t1, t2, y_head

In [16]:

# str_sents = '他说明天要去钓鱼, 在中山公园. 您的快递到了'
str_sents = '您好, 您的快递到了'


t1, t2, y_head = str_felling_detect(model, str_sents)

t1, t2, y_head

(0.6539220809936523, 0.34607794880867004, 0)

In [23]:
test_dataset = Dataset('train')

print(len(test_dataset))

Loading cached processed dataset at /home/mylady/code/python/DL-pytorch/apps/huggingface/data/ChnSentiCorp/train/cache-2c893bcab2dc48fd.arrow


9552


In [24]:


sum_total = 0
sum_acc_count = 0

for i, str_sents in enumerate(test_dataset):
    
    input_str = str_sents[0] + str_sents[1]
    label = str_sents[2]
    
    # print(i, input_str, label)
    
    t1, t2, y_head = str_felling_detect(model, input_str)
    
    if y_head == label:
        sum_acc_count += 1
    else:
        print('error:预测:%s 实际label: %s,  %.4f %.4f ' % (y_head, label, t1, t2))
        print("input_str_1: %s" % str_sents[0])
        print("input_str_2: %s" % str_sents[1])
        print("")
        pass

    sum_total += 1
    if i >= 50:
        break


# 1: 不相关 0:相关
print('acc: %.4f' % (sum_acc_count / sum_total))

error:预测:1 实际label: 0,  0.0002 0.9998 
input_str_1: 15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜
input_str_2: 欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错

error:预测:1 实际label: 0,  0.0000 1.0000 
input_str_1: 1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来.
input_str_2:  3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办.

error:预测:1 实际label: 0,  0.0009 0.9991 
input_str_1: 呵呵，虽然表皮看上去不错很精致，但是我还是能看得出来是盗的
input_str_2: 。但是里面的内容真的不错，我妈爱看，我自己也学着找一些穴位。

error:预测:1 实际label: 0,  0.0004 0.9996 
input_str_1: 这本书实在是太烂了,以前听浙大的老师说这本书怎么怎么不对,哪些地方都是
input_str_2: 误导的还不相信,终于买了一本看一下,发现真是~~~无语,这种书都写得出来

error:预测:1 实际label: 0,  0.0012 0.9988 
input_str_1: 我看过朋友的还可以，但是我订的书迟迟未到已有半个月，都没有收
input_str_2: 到打电话也没有用，以后你们订书一定要考虑好！当当实在是太慢了

error:预测:1 实际label: 0,  0.0019 0.9981 
input_str_1: 送的内胆包有点不好，还有外接电源中间连接处无法全部插入。
input_str_2: 续航时间也没有标称的那么长，希望京东能注意宣传的真实性。

error:预测:1 实际label: 0,  0.0000 1.0000 
input_str_1: 这是我第1次给全五星哦^_^超级快!这是最快收到书的一次了.我是中午的时候订的,结果第2天上午就收到了,算了一下,1天的时间都还没到呢!在此,感激下当当的服务.
input_str_2: ..我的确是很急需这本书呢.关于书的本身,也很不错.内容还是很丰富的,值得推荐,对于训练和培养逻辑思维套式有一定的帮助,推荐一下~还有祝朋友们都面试成功,哈哈哈

In [ ]:
# 保存模型

# 保存

model_save_path = 'chinese_infer_mission_2023_4_10.pt'
# torch.save(model.state_dict(),  model_save_path)  # 推荐的文件后缀名是pt或pth